# Last FM hometask <br>
https://www.kaggle.com/ravichaubey1506/lastfm <br>
<ol>
<li>Выбрать данные по странам своей группы (совместно): <br>
    3530203_70101: Germany, Netherlands <br>
    3530203_70102: Belarus, Ukraine, Poland, Russian Federation<br>
    3530903_70301: Sweden, Finland, Norway, Denmark, Iceland<br>
    3530903_70302: Spain, Portugal, France, Italy, Belgium<br>
<li>Попытаться найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Хотя бы 5 правил.
<li>Вывести эти правила в отдельных ячейках. 
<li>Подумать, как можно было бы использовать полученные правила.
</ol>

In [1]:
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth, fpmax
import pandas as pd

In [2]:
data = pd.read_csv("lastfm.csv")
data.head()

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany


In [3]:
countries = ['Spain', 'Portugal', 'France', 'Italy', 'Belgium']
data = data.loc[data['country'].isin(countries)]
data = data.drop(data[data['artist']=='[unknown]'].index)
data

,user,artist,sex,country
83,6,lily allen,m,Portugal
84,6,kanye west,m,Portugal
85,6,sigur rós,m,Portugal
86,6,pink floyd,m,Portugal
87,6,stevie wonder,m,Portugal
...,...,...,...,...
289758,19707,the decemberists,m,Spain
289759,19707,my bloody valentine,m,Spain
289760,19707,bloc party,m,Spain
289761,19707,new order,m,Spain


In [4]:
complex_data = data.groupby(['user', 'sex', 'country'])['artist'].apply(lambda x: ';'.join(x)).reset_index()
complex_data

,user,sex,country,artist
0,6,m,Portugal,lily allen;kanye west;sigur rós;pink floyd;ste...
1,12,m,Italy,queen;the beatles;lynyrd skynyrd;neil young;th...
2,36,f,Italy,m.i.a.;fugazi;johnny cash;clint mansell;elton ...
3,47,m,Italy,pj harvey;radiohead;zero 7;einstürzende neubau...
4,74,m,Spain,in extremo;nine inch nails;billy talent;panic!...
...,...,...,...,...
1412,19690,m,Italy,pink floyd;the beatles;the white stripes;depec...
1413,19693,m,Belgium,air;simon & garfunkel;pixies;serge gainsbourg;...
1414,19697,m,Belgium,underworld;pendulum;the prodigy;sufjan stevens...
1415,19704,m,Italy,keane;pinback;coldplay;travis;calexico;doves;d...


In [5]:
complex_binary_data = pd.concat([
    complex_data.drop(['country', 'sex', 'artist'], axis=1), 
    pd.get_dummies(complex_data['sex']), 
    pd.get_dummies(complex_data['country']), 
    complex_data['artist'].str.get_dummies(';')
    ], axis = 1).set_index(['user'])
complex_binary_data

,f,m,Belgium,France,Italy,Portugal,Spain,...and you will know us by the trail of dead,2pac,3 doors down,...,wilco,within temptation,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,yo la tengo,zero 7,Édith piaf
user,,,,,,,,,,,,,,,,,,,,,
6,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
74,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19690,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19693,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19697,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
frequent_itemsets_apriori = apriori(complex_binary_data, min_support=0.015, use_colnames=True)
frequent_itemsets_apriori

,support,itemsets
0,0.220183,(f)
1,0.779817,(m)
2,0.112915,(Belgium)
3,0.230769,(France)
4,0.214538,(Italy)
...,...,...
2484,0.016937,"(placebo, radiohead, coldplay, muse)"
2485,0.016231,"(the killers, radiohead, coldplay, muse)"
2486,0.017643,"(the killers, the strokes, coldplay, muse)"
2487,0.016231,"(the killers, interpol, radiohead, muse)"


In [7]:
frequent_itemsets_fpgrowth = fpgrowth(complex_binary_data, min_support=0.015, use_colnames=True)
frequent_itemsets_fpgrowth

,support,itemsets
0,0.779817,(m)
1,0.174312,(muse)
2,0.130558,(pink floyd)
3,0.123500,(metallica)
4,0.096683,(depeche mode)
...,...,...
2484,0.015526,"(deftones, Spain)"
2485,0.016231,"(deftones, the smashing pumpkins)"
2486,0.015526,"(deftones, muse)"
2487,0.017643,"(m, kreator)"


In [8]:
rules_lift_apriori = association_rules(frequent_itemsets_apriori, min_threshold=0.015, metric="confidence")
rules_lift_apriori

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(f),(Belgium),0.220183,0.112915,0.022583,0.102564,0.908333,-0.002279,0.988467
1,(Belgium),(f),0.112915,0.220183,0.022583,0.200000,0.908333,-0.002279,0.974771
2,(France),(f),0.230769,0.220183,0.043049,0.186544,0.847222,-0.007763,0.958647
3,(f),(France),0.220183,0.230769,0.043049,0.195513,0.847222,-0.007763,0.956175
4,(f),(Italy),0.220183,0.214538,0.052929,0.240385,1.120477,0.005691,1.034026
...,...,...,...,...,...,...,...,...,...
8511,"(radiohead, muse)","(interpol, the strokes)",0.080452,0.033874,0.016231,0.201754,5.955958,0.013506,1.210311
8512,(interpol),"(the strokes, muse, radiohead)",0.079746,0.021171,0.016231,0.203540,9.613864,0.014543,1.228974
8513,(the strokes),"(interpol, radiohead, muse)",0.087509,0.028229,0.016231,0.185484,6.570766,0.013761,1.193066
8514,(radiohead),"(interpol, the strokes, muse)",0.237826,0.023289,0.016231,0.068249,2.930582,0.010693,1.048254


In [9]:
rules_lift_fpgrowth = association_rules(frequent_itemsets_fpgrowth, min_threshold=0.015, metric="confidence")
rules_lift_fpgrowth

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(m),(muse),0.779817,0.174312,0.129146,0.165611,0.950083,-0.006785,0.989572
1,(muse),(m),0.174312,0.779817,0.129146,0.740891,0.950083,-0.006785,0.849771
2,(Spain),(muse),0.355681,0.174312,0.067749,0.190476,1.092732,0.005749,1.019968
3,(muse),(Spain),0.174312,0.355681,0.067749,0.388664,1.092732,0.005749,1.053952
4,(f),(muse),0.220183,0.174312,0.045166,0.205128,1.176788,0.006785,1.038769
...,...,...,...,...,...,...,...,...,...
8511,(muse),(deftones),0.174312,0.035286,0.015526,0.089069,2.524211,0.009375,1.059042
8512,(m),(kreator),0.779817,0.019054,0.017643,0.022624,1.187364,0.002784,1.003653
8513,(kreator),(m),0.019054,0.779817,0.017643,0.925926,1.187364,0.002784,2.972477
8514,(stereophonics),(coldplay),0.021171,0.188426,0.016231,0.766667,4.068789,0.012242,3.478173


<h1>Правило 1</h1>
Давайте выясним мужчинам или женщинам больше нравится любимая группа Валерия Ковшова - Muse:

In [10]:
def m_or_f(rules_data):
    return rules_data[
        (rules_data['consequents'] == {'muse'}) 
        & ((rules_data['antecedents'] == {'m'}) | (rules_data['antecedents'] == {'f'}))
    ].sort_values(by='lift', ascending=False)

In [11]:
m_or_f(rules_lift_apriori)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
56,(f),(muse),0.220183,0.174312,0.045166,0.205128,1.176788,0.006785,1.038769
474,(m),(muse),0.779817,0.174312,0.129146,0.165611,0.950083,-0.006785,0.989572


In [12]:
m_or_f(rules_lift_fpgrowth)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,(f),(muse),0.220183,0.174312,0.045166,0.205128,1.176788,0.006785,1.038769
0,(m),(muse),0.779817,0.174312,0.129146,0.165611,0.950083,-0.006785,0.989572


Смотрим, анализируем:
<li>supp - Мужчин слушающих Muse в датасете явно больше, чем женщин, но это не о чём не говорит, так как в датасете мужчин сильно больше, поэтому смотрим дальше:
<li>conf - И тут мы уже видим, что правило Женщины слушают Muse чаще срабатывает для нашего датасет (попод задуматься)
<li>lify - Смотрим на отношение "зависимости к независимости" для набора пол и группа Muse и видим, что, у женщин lift > 1, что окончательно даёт нам понимание, что правило "Женщины слушают Muse" сильнее, чем правило "Мужчины слушают Muse" =(

<h1>Правило 2</h1>
Теперь давайте выясним, что можно послушать ещё, если вам нравится Muse:

In [13]:
def what_else_if_i_love_muse(rules_data):
    return rules_data[
        (rules_data['antecedents'] == {'muse'})
        & (rules_data['consequents'].map(
            lambda x: 
            ('m' not in x) & ('f' not in x)
            & ('Spain' not in x) & ('Portugal' not in x) & ('France' not in x) & ('Italy' not in x) & ('Belgium' not in x)))
    ].sort_values(by='lift', ascending=False).head(3)

In [14]:
what_else_if_i_love_muse(rules_lift_apriori)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
6721,(muse),"(interpol, oasis)",0.174312,0.019760,0.016231,0.093117,4.712406,0.012787,1.08089
8501,(muse),"(the killers, interpol, radiohead)",0.174312,0.019760,0.016231,0.093117,4.712406,0.012787,1.08089
8277,(muse),"(arctic monkeys, the strokes, coldplay)",0.174312,0.019054,0.015526,0.089069,4.674464,0.012204,1.07686


In [15]:
what_else_if_i_love_muse(rules_lift_fpgrowth)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
6753,(muse),"(the killers, interpol, radiohead)",0.174312,0.019760,0.016231,0.093117,4.712406,0.012787,1.08089
6785,(muse),"(interpol, oasis)",0.174312,0.019760,0.016231,0.093117,4.712406,0.012787,1.08089
2553,(muse),"(arctic monkeys, the strokes, coldplay)",0.174312,0.019054,0.015526,0.089069,4.674464,0.012204,1.07686


Смотрим, анализируем и добавляем в плейлист (если ещё не сделали этого):
<li>oasis
<li>the killers
<li>interpol
<li>radiohead
<li>the strokes
<li>arctic monkeys
<li>coldplay

<h1>Правило 3</h1>
Давайте выясним, а где же искать единомышленников за рубежом:

In [16]:
def where_else_love_muse(rules_data):
    return rules_data[
        (rules_data['antecedents'] == {'muse'})
        & (
            (rules_data['consequents'] == {'Spain'})
            | (rules_data['consequents'] == {'Portugal'})
            | (rules_data['consequents'] == {'France'})
            | (rules_data['consequents'] == {'Italy'})
            | (rules_data['consequents'] == {'Belgium'}))
    ].sort_values(by='lift', ascending=False)

In [17]:
where_else_love_muse(rules_lift_apriori)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
867,(muse),(Portugal),0.174312,0.086097,0.018349,0.105263,1.222606,0.003341,1.021421
983,(muse),(Spain),0.174312,0.355681,0.067749,0.388664,1.092732,0.005749,1.053952
735,(muse),(France),0.174312,0.230769,0.038814,0.222672,0.964912,-0.001411,0.989583
817,(muse),(Italy),0.174312,0.214538,0.033874,0.194332,0.905817,-0.003522,0.974920
689,(muse),(Belgium),0.174312,0.112915,0.015526,0.089069,0.788816,-0.004157,0.973823


In [18]:
where_else_love_muse(rules_lift_fpgrowth)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
795,(muse),(Portugal),0.174312,0.086097,0.018349,0.105263,1.222606,0.003341,1.021421
3,(muse),(Spain),0.174312,0.355681,0.067749,0.388664,1.092732,0.005749,1.053952
7,(muse),(France),0.174312,0.230769,0.038814,0.222672,0.964912,-0.001411,0.989583
13,(muse),(Italy),0.174312,0.214538,0.033874,0.194332,0.905817,-0.003522,0.974920
3575,(muse),(Belgium),0.174312,0.112915,0.015526,0.089069,0.788816,-0.004157,0.973823


Смотрим, анализируем и берём билеты в Португалию, а группе Muse подсказываем, что там можно продать больше всего билетов

<h1>Правило 4</h1>
В наших правилах учавствовали такие метрики как supp, conf и lify, но мы пока не использовали conviction, которая показывает частотность ошибок нашего правила. Чем результат выше 1, тем лучше, а если результат меньше 1, то наше правило ближе к исключению чем к правилу.
Давайте на основе этой метрики выясним, в какой стране мужчинам больше нравится Muse:

In [19]:
def where_f_love_muse(rules_data):
    return rules_data[
        (rules_data['antecedents'] == {'muse'})
        & (rules_data['consequents'].map(lambda x: ('m' in x) & (repr(x).count(',') == 1)))
        & ((rules_data['consequents'].map(lambda x: ('Spain' in x) | ('Portugal' in x) | ('France' in x) | ('Italy' in x) | ('Belgium' in x))))
    ].sort_values(by='conviction', ascending=False).head(3)

In [20]:
where_f_love_muse(rules_lift_apriori)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2965,(muse),"(m, Spain)",0.174312,0.273112,0.049400,0.283401,1.037672,0.001793,1.014358
2545,(muse),"(France, m)",0.174312,0.187721,0.030346,0.174089,0.927384,-0.002376,0.983495
2701,(muse),"(m, Italy)",0.174312,0.161609,0.023289,0.133603,0.826707,-0.004882,0.967676


In [21]:
where_f_love_muse(rules_lift_fpgrowth)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
19,(muse),"(Spain, m)",0.174312,0.273112,0.049400,0.283401,1.037672,0.001793,1.014358
89,(muse),"(France, m)",0.174312,0.187721,0.030346,0.174089,0.927384,-0.002376,0.983495
141,(muse),"(m, Italy)",0.174312,0.161609,0.023289,0.133603,0.826707,-0.004882,0.967676


Смотрим, анализируем и если вам не нравится Португалия, то берём билеты в Испанию, ведь любители корриды и фламенко так же как и мы любят Muse <3

<h1>Правило 5</h1>
Теперь, когда мы знаем, что у Португальцев и Испанцев в целом смежный с нами вкус, давайте посмотрим, что ещё им нарвится, вдруг мы найдём что-то новое и свежнее для наших плейлистов:

In [22]:
def spain_and_portugal_love (rules_data):
    return rules_data[
        ((rules_data['antecedents'] == {'Spain'}) | (rules_data['antecedents'] == {'Portugal'}))
        & (rules_data['consequents'].map(lambda x: (repr(x).count(',') == 1) & ('muse' in x) & ('m' not in x) & ('f' not in x))
        & (rules_data['consequents'] != {'muse'}))
    ].sort_values(by='lift', ascending=False).head(5)

In [23]:
spain_and_portugal_love(rules_lift_apriori)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
5712,(Spain),"(the killers, muse)",0.355681,0.049400,0.026817,0.075397,1.526247,0.009247,1.028117
5639,(Spain),"(foo fighters, muse)",0.355681,0.033169,0.017643,0.049603,1.495483,0.005845,1.017292
5658,(Spain),"(franz ferdinand, muse)",0.355681,0.037403,0.019054,0.053571,1.432278,0.005751,1.017084
5717,(Spain),"(the strokes, muse)",0.355681,0.038109,0.019054,0.053571,1.405754,0.005500,1.016338
5675,(Spain),"(interpol, muse)",0.355681,0.040932,0.019054,0.053571,1.308805,0.004496,1.013355


In [24]:
spain_and_portugal_love(rules_lift_fpgrowth)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3662,(Spain),"(the killers, muse)",0.355681,0.049400,0.026817,0.075397,1.526247,0.009247,1.028117
5265,(Spain),"(foo fighters, muse)",0.355681,0.033169,0.017643,0.049603,1.495483,0.005845,1.017292
5922,(Spain),"(franz ferdinand, muse)",0.355681,0.037403,0.019054,0.053571,1.432278,0.005751,1.017084
2393,(Spain),"(the strokes, muse)",0.355681,0.038109,0.019054,0.053571,1.405754,0.005500,1.016338
6637,(Spain),"(interpol, muse)",0.355681,0.040932,0.019054,0.053571,1.308805,0.004496,1.013355


Смотрим, анализируем и видим знакомые имемна и названия. Кажется в правиле два мы уже добавили всех, кроме the fighters и franz ferdinand - давайте дадим шанс и им и добавим их в плейлист =)